Ejercicio 1

In [1]:
import pandas as pd
import numpy as np


datos_ext='base_ini_externa.txt'
datos_int='base_ini_interna.csv'
datos_pt1='base_ini_parte1.csv'
datos_comp3='base_comportamiento_3.csv'
datos_comp2='base_comportamiento_2.txt'
datos_inicio='bd_inicio_parte1.txt'
datos_inv='base_inversion.txt'

b_comp3=pd.read_csv(datos_comp3)
b_comp2=pd.read_csv(datos_comp2,delimiter=' ')
b_inicio=pd.read_csv(datos_inicio,delimiter=' ')
b_int=pd.read_csv(datos_int)
b_ext=pd.read_csv(datos_ext,delimiter=' ')
b_pt1=pd.read_csv(datos_pt1)
b_inv=pd.read_csv(datos_inv,delimiter=' ')


#print(b_comp3.head(2))
#print(b_comp2.head(2))
#print(b_inicio.head(2),'\n',b_int.head(2),'\n',b_ext.head(2),'\n',b_pt1.head(2),'\n',b_inv.head(2))


unir las bases de las variables

In [2]:
merge_t = b_inicio.merge( b_int, on="_id_").merge(b_ext, on='_id_').merge(b_pt1, on='_id_').merge(b_inv,on='_id2_') #unimos las bases
merge_t['num_caso']=merge_t['_id_']  
merge_t=merge_t.set_index('num_caso') #agregamos los id como indices
#merge_t=merge_t.dropna()
merge_t=merge_t.sort_values(by=['_id_'])   #ordenamos por inidces

merge_t.head(2)

,_id_,comp_interno4_x,fecha_inicio,comp_interno1,comp_interno2,comp_interno3,comp_interno4_y,categoria1,categoria2,categoria3,comp_externo1,comp_externo2,comp_externo3,comp_externo4,_id2_,comp_mixto1,inversion
num_caso,,,,,,,,,,,,,,,,,
1,1,67.0,2016-03-31,3039.513678,b'1',26.0,67.0,3.0,3.0,171.0,217.391304,21.545533,102.0,-230.0,23,0.3800,52.50049
7,7,75.0,2016-01-19,1547.112462,b'1',28.0,75.0,1.0,11.0,173.0,468.115942,21.674806,108.0,-230.0,161,0.3336,0.04748


unir las bases del comportamiento

In [3]:
comportamiento=pd.merge(b_comp3,b_comp2, how='outer', left_index=True, right_index=True)  #unimos las bases de comportamiento
comportamiento=comportamiento.drop(['mes_informacion_y'], axis=1)     #eliminamos las columnas repetidas
comportamiento=comportamiento.rename(columns={"num_caso_x": "_id_",'num_caso_y':'num_caso'})  #renombramos


In [4]:
comportamiento.head()

,_id_,mes_informacion_x,comportamiento_j,num_caso,Tiempo_aparicion
0,1,2016-04-01 00:00:00,0.0,1,0.0
1,1,2016-05-01 00:00:00,0.0,1,1.0
2,1,2016-06-01 00:00:00,0.0,1,2.0
3,1,2016-07-01 00:00:00,0.0,1,3.0
4,1,2016-08-01 00:00:00,0.0,1,4.0


In [5]:
comp=comportamiento.groupby(['num_caso']) #agrupar el comportamiento por num caso
#hacemos un data frame que contenga el numero de atrasos entre días, si el cociente es 1 significa que no pagó
pr=pd.DataFrame({'comp':comp['comportamiento_j'].sum()/comp['Tiempo_aparicion'].sum()})
pr['_id_']=pr.index #agregamos la columna _id_ para cruzarla con las demas tablas
pr.head()

,comp,_id_
num_caso,,
1,0.003702,1
7,0.059113,7
8,0.002992,8
9,0.000000,9
10,0.008333,10


In [31]:
n=0.1
len(pr.loc[pr['comp']>=n]) #personas que no pagarón o se atrasaron mucho, entre más cerca de 1 es que no paga

1492

In [11]:
pr=pr.sort_values(by=['_id_'])#ordenar por id
pr['_id_']=pr['_id_'].astype(int)
pr.tail()

,comp,_id_
num_caso,,
98987,0.0,98987
98988,0.0,98988
98990,0.0,98990
98992,0.0,98992
98997,0.0,98997


In [12]:
pr['pr_bin']=pr['comp']                   #copiamos la columna
pr.loc[pr['comp']>=n,'pr_bin']=0    # filtramos con 0 o 1 según su cociente de pr['comp'], 0 no paga
pr.loc[pr['comp']<n,'pr_bin']=1     #1 si paga

In [13]:
tab=merge_t.merge(pr,on='_id_', how="right")   #unimos promedio de atraso y las variales

In [36]:
len(tab)

15379

In [39]:
tab=tab.dropna(how='any')
cols=tab.columns

x=tab.loc[:,[cols[1],cols[3],cols[5],cols[7],cols[8],cols[9],cols[10],cols[11],cols[12],cols[13],cols[14], cols[15]]]
y=tab['pr_bin']

15379


In [16]:
cols

Index(['_id_', 'comp_interno4_x', 'fecha_inicio', 'comp_interno1',
       'comp_interno2', 'comp_interno3', 'comp_interno4_y', 'categoria1',
       'categoria2', 'categoria3', 'comp_externo1', 'comp_externo2',
       'comp_externo3', 'comp_externo4', '_id2_', 'comp_mixto1', 'inversion',
       'comp', 'pr_bin'],
      dtype='object')

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [19]:
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.Logit(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary())

Optimization terminated successfully.
         Current function value: 0.085673
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                 pr_bin   No. Observations:                12303
Model:                          Logit   Df Residuals:                    12290
Method:                           MLE   Df Model:                           12
Date:                Wed, 05 Jan 2022   Pseudo R-squ.:                 0.03411
Time:                        14:50:20   Log-Likelihood:                -1054.0
converged:                       True   LL-Null:                       -1091.3
Covariance Type:            nonrobust   LLR p-value:                 4.675e-11
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0006   3.84e+06    1.6e-10      1.000   -7.53e+06    7.53e+06
comp_interno4

In [32]:
x=tab.loc[:,[cols[3],cols[8],cols[11],cols[12], cols[15]]]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)


In [33]:
cols[10]

'comp_externo1'

In [34]:
X_test = sm.add_constant(X_test, prepend=True)
predicciones = modelo.predict(exog = X_test)
clasificacion = np.where(predicciones<0.5, 0, 1)
accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = clasificacion,
            normalize = True
           )
print("")
print(f"El accuracy de test es: {100*accuracy}%")


El accuracy de test es: 98.63459037711313%


In [35]:
modelo=LogisticRegression(C=1,solver='newton-cg',max_iter=1000).fit(X_train,y_train)
print(modelo.coef_)
print(modelo.intercept_)
print(modelo.score(X_test,y_test))

[[ 3.39367632e-06  1.67159186e-02 -2.79875890e-03  1.95326328e-03
  -1.32772006e+00]]
[4.27950573]


C:\Users\aldai\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\aldai\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


ValueError: X has 6 features per sample; expecting 5

In [24]:
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.Logit(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary())

Optimization terminated successfully.
         Current function value: 0.087960
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                 pr_bin   No. Observations:                12303
Model:                          Logit   Df Residuals:                    12297
Method:                           MLE   Df Model:                            5
Date:                Wed, 05 Jan 2022   Pseudo R-squ.:                0.008328
Time:                        14:51:23   Log-Likelihood:                -1082.2
converged:                       True   LL-Null:                       -1091.3
Covariance Type:            nonrobust   LLR p-value:                  0.002733
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.9340      0.743      7.981      0.000       4.477       7.391
comp_interno1  1.642